In [ ]:
# 通过POST请求获取中国石化所有关键字为年报的搜索结果的网页源代码
import requests 
import re #导入模块
from bs4 import BeautifulSoup#导入模块
headers={"User-Agent":"Mozilla/5.0"}
params={'prodType':'600028','keyWord':'年报'}
r=requests.post(url="http://eid.csrc.gov.cn/101111/index_f.html",headers=headers,data=params)
r_text=r.text
print(r_text)
#从获取的源代码可以看出，已经获取了中国石化的定期报告搜索结果

In [ ]:
#Step1. 通过post请求获取单个公司指定报告的搜索页面（有翻页的情况可以通过调整网址来实现
#Step2. 解析搜索页面，获取每个报告的pdf所在网址，据此下载单个公司所有的报告
#Step3. 通过循环可下载多个公司的指定报告 （rp.requests.get(url) with open(path,"wb+") as f: f.write(rp.content)）

In [ ]:
# 处理翻页
## 观察网页变化可以发现，搜索结果一共4页，而目前只获取了第1页的链接信息
## 需要考虑两方面：1.观察网站翻页后网址的变化规律；2.观察翻页后POST数据的变化规律
## 发现翻页后POST信息不变而网址变化（如第2页的post_url为 http://eid.csrc.gov.cn/101111/index_2_f.html）
## 考虑通过对网址进行循环来获取所有页面的信息。需要确定的一个关键参数是，最大页数，可以用正则表达式提取
 
max_page_text=re.search(r"共<b>(\d+)</b>页",r_text).group(1)
#Step 1：通过子表达式提取页数的字符串
max_page=int(max_page_text)
#Step 2：将页数转化为整数型
print(max_page)

In [ ]:
max_page_text=re.search(r"共<b>(\d+)</b>页",r_text)
print(max_page)

In [ ]:
# 综合以上内容，定义一个函数来下载单个公司的所有指定关键字报告

#根据指定的证券代码SecCode和关键字keyword下载证监会官网上公司的报告
#将下载的报告保存在公司证券代码为名的文件夹中
import os

def get_reports(SecCode,keyword):
    path=SecCode
    if not os.path.exists(path):#当以证券代码为名的文件夹不存在时，则创建一个
        os.mkdir(path)
    params={'prodType':SecCode,'keyWord':keyword}
    headers={"User-Agent":"Mozilla/5.0"}
    r=requests.post(url="http://eid.csrc.gov.cn/101111/index_f.html",headers=headers,data=params)
    r_text=r.text
    max_page_text=re.search("共<b>(\d+)</b>页",r_text).group(1)
    max_page=int(max_page_text)
    for pg_num in range(1,max_page+1):
        r=requests.post(url="http://eid.csrc.gov.cn/101111/index_"+str(pg_num)+"_f.html",headers=headers,data=params)
        r_text=r.text
        bs=BeautifulSoup(r_text,'html.parser')
    # pdf的下载链接包含在以td为名的，属性width的值为"400px"的标签对象中
        link_list=bs.find_all('td',attrs={"width":"400px;"}) 
        links=[(lk['onclick'].split("'"))[1] for lk in link_list]
    #具体的链接包含在标签对象的onclick属性的值中
        for item in links: 
            filename =item.split("/")[-1]
    #取下载链接的最后一部分作为文件名
            print("downloading:"+filename)
    #打印下载进度
            rp=requests.get(item)
            with open(path+"/%s"%(filename),"wb+") as f:
                f.write(rp.content)

In [ ]:
get_reports('600028','年报')